In [ ]:
import tensorflow as tf

print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

In [ ]:
import csv, pandas
from nobrainer.io import read_csv


all_csv = '/workspace/tf/HDDLinux/Stanford/data/mriqc-shared/experiments/experiment_defaced_reproduce/128/csv_F15/all.csv'
prepend = '/workspace/tf'
all_paths = read_csv(all_csv)
print(all_paths)
new_paths = []
for p in all_paths:
    new_paths.append((os.path.join(prepend, p[0].split('shank/')[1]), p[1]))
    

In [ ]:
print(new_paths)
with open('/workspace/tf/HDDLinux/Stanford/data/mriqc-shared/experiments/experiment_defaced_reproduce/128/csv_F15/new_all.csv','w') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(['X','Y'])
    for row in new_paths:
        csv_out.writerow(row)

In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from nobrainer.io import read_csv
all_csv = '/workspace/tf/HDDLinux/Stanford/data/mriqc-shared/experiments/experiment_defaced_reproduce/128/csv_F15/new_all.csv'
all_paths = read_csv(all_csv)

X, Y = list(zip(*all_paths))
X, Y = np.array(X), np.array(Y)

base_path = '/workspace/tf/HDDLinux/Stanford/data/mriqc-shared/experiments/experiment_defaced_reproduce/128/csv_F15'

skf = StratifiedKFold(n_splits=15, random_state=42, shuffle=True)

i = 1
for train_index, test_index in skf.split(X, Y):
    
    os.makedirs(os.path.join(base_path, f'train_test_fold_{i}'), exist_ok=True)
    path = os.path.join(base_path, f'train_test_fold_{i}')
    tpath = os.path.join(path, 'training.csv')
    vpath = os.path.join(path, 'validation.csv')
    
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    
    with open(tpath,'w') as out:
        csv_out=csv.writer(out)
        csv_out.writerow(['X','Y'])
        for row in list(zip(X_train, y_train)):
            csv_out.writerow(row)
    
    with open(vpath,'w') as out:
        csv_out=csv.writer(out)
        csv_out.writerow(['X','Y'])
        for row in list(zip(X_test, y_test)):
            csv_out.writerow(row)
            
    i+= 1
    

In [ ]:
import os, nobrainer, csv, random
from nobrainer.io import read_csv
from nobrainer.tfrecord import write as write_tfrecord

base_path = '/workspace/tf/HDDLinux/Stanford/data/mriqc-shared/experiments/experiment_defaced_reproduce/128/'

for fold in range(1, 16):
    csv_path = os.path.join(base_path, f'csv_F15/train_test_fold_{fold}')
    tcsv_path = os.path.join(csv_path, 'training.csv')
    vcsv_path = os.path.join(csv_path, 'validation.csv')

    tfrecords_path = os.path.join(base_path, f'tfrecords_F15_shuffled_2/train_test_fold_{fold}')

    os.makedirs(tfrecords_path, exist_ok=True)

    num_parallel_calls = 10

    t_filepaths = read_csv(tcsv_path)
    v_filepaths = read_csv(vcsv_path)
    
    random.shuffle(t_filepaths)
    random.shuffle(v_filepaths)
    
    write_tfrecord(
            features_labels=t_filepaths,
            filename_template=os.path.join(tfrecords_path, 'data-train_shard-{shard:03d}.tfrec'),
            examples_per_shard=3,
            processes=num_parallel_calls,
    )
    
    write_tfrecord(
            features_labels=v_filepaths,
            filename_template=os.path.join(tfrecords_path, 'data-valid_shard-{shard:01d}.tfrec'),
            examples_per_shard=1,
            processes=num_parallel_calls,
    )


In [ ]:
"""Training module for nondefaced-detector."""


import os
import math

import tensorflow as tf
import pandas as pd
import numpy as np

from sklearn.utils import class_weight
from tensorflow.keras import backend as K
from tensorflow.keras import metrics
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping

import sys
sys.path.append('../')
from nondefaced_detector.models import model as _model
from nondefaced_detector.dataloaders.dataset import get_dataset


def scheduler(epoch):
    if epoch < 3:
        return 0.001
    else:
        return 0.001 * tf.math.exp(0.1 * (10 - epoch))


def train(
    csv_path,
    model_save_path,
    tfrecords_path,
    volume_shape=(128, 128, 128),
    image_size=(128, 128),
    dropout=0.2,
    batch_size=16,
    n_classes=2,
    n_epochs=15,
    mode="CV",
):
    """Train a model.

    Parameters
    ----------
    csv_path: str - Path
        Path to the csv file containing training volume paths, labels (X, Y).
    model_save_path: str - Path
        Path to where the save model and model weights.
    tfrecords_path: str - Path
        Path to preprocessed training tfrecords.
    volume_shape: tuple of size 3, optional, default=(128, 128, 128)
        The shape of the preprocessed volumes.
    image_size: tuple of size 2, optional, default=(128, 128)
        The shape of a 2D slice along each volume axis.
    dropout: float, optional, default=0.4
         Float between 0 and 1. Fraction of the input units to drop.
    batch_size: int, optional, default=16
        No. of training examples utilized in each iteration.
    n_classes: int, optional, default=2
        No. of unique classes to train the model on. Default assumption is a
        binary classifier.
    n_epochs: int, optional, default=15
        No. of complete passes through the training dataset.
    mode: str, optional, default=15
        One of "CV" or "full". Indicates the type of training to perform.

    Returns
    -------
    `tf.keras.callbacks.History`
        A History object that records several metrics such as training/validation loss/metrics
        at successive epochs.
    """
    
    tf.keras.backend.clear_session()
    
    train_csv_path = os.path.join(csv_path, "training.csv")
    train_paths = pd.read_csv(train_csv_path)["X"].values
    train_labels = pd.read_csv(train_csv_path)["Y"].values

    if mode == "CV":
        valid_csv_path = os.path.join(csv_path, "validation.csv")
        valid_paths = pd.read_csv(valid_csv_path)["X"].values
        valid_labels = pd.read_csv(valid_csv_path)["Y"].values

    weights = class_weight.compute_class_weight(
        "balanced", np.unique(train_labels), train_labels
    )
    weights = dict(enumerate(weights))

    planes = ["axial", "sagittal", "coronal", "combined"]

    global_batch_size = batch_size

    os.makedirs(model_save_path, exist_ok=True)
    cp_save_path = os.path.join(model_save_path, "weights")
    logdir_path = os.path.join(model_save_path, "tb_logs")
    metrics_path = os.path.join(model_save_path, "metrics")

    os.makedirs(metrics_path, exist_ok=True)

    for plane in planes:

        logdir = os.path.join(logdir_path, plane)
        os.makedirs(logdir, exist_ok=True)

        tbCallback = TensorBoard(log_dir=logdir)

        os.makedirs(os.path.join(cp_save_path, plane), exist_ok=True)

        model_checkpoint = ModelCheckpoint(
            os.path.join(cp_save_path, plane, "best-wts.h5"),
            monitor="val_loss",
            save_weights_only=True,
            mode="min",
        )

        if not plane == "combined":
            lr = 1e-3
            model = _model.Submodel(
                root_path=cp_save_path,
                input_shape=image_size,
                dropout=dropout,
                name=plane,
                include_top=True,
                weights=None,
            )
        else:
            lr = 5e-4
            model = _model.CombinedClassifier(
                input_shape=image_size,
                dropout=dropout,
                trainable=True,
                wts_root=cp_save_path,
            )

        print("Submodel: ", plane)

        METRICS = [
            metrics.TruePositives(name="tp"),
            metrics.FalsePositives(name="fp"),
            metrics.TrueNegatives(name="tn"),
            metrics.FalseNegatives(name="fn"),
            metrics.BinaryAccuracy(name="accuracy"),
            metrics.Precision(name="precision"),
            metrics.Recall(name="recall"),
            metrics.AUC(name="auc"),
        ]

        model.compile(
            loss=tf.keras.losses.binary_crossentropy,
            optimizer=Adam(learning_rate=lr),
            metrics=METRICS,
        )

        dataset_train = get_dataset(
            file_pattern=os.path.join(tfrecords_path, "data-train_*"),
            n_classes=n_classes,
            batch_size=global_batch_size,
            volume_shape=volume_shape,
            plane=plane,
            n_slices=80,
            shuffle_buffer_size=global_batch_size,
        )

        steps_per_epoch = math.ceil(len(train_paths) / batch_size)

        if mode == "CV":
            earlystopping = EarlyStopping(monitor="val_accuracy", patience=3)

            dataset_valid = get_dataset(
                file_pattern=os.path.join(tfrecords_path, "data-valid_*"),
                n_classes=n_classes,
                batch_size=global_batch_size,
                volume_shape=volume_shape,
                plane=plane,
                n_slices=80,
                shuffle_buffer_size=global_batch_size,
            )

            validation_steps = math.ceil(len(valid_paths) / batch_size)

            history = model.fit(
                dataset_train,
                epochs=n_epochs,
                steps_per_epoch=steps_per_epoch,
                validation_data=dataset_valid,
                validation_steps=validation_steps,
                callbacks=[tbCallback, model_checkpoint, earlystopping],
                class_weight=weights,
            )

            hist_df = pd.DataFrame(history.history)

        else:
#             earlystopping = EarlyStopping(monitor="loss", patience=3)
            print(model.summary())
            print("Steps/Epoch: ", steps_per_epoch)
            history = model.fit(
                dataset_train,
                epochs=n_epochs,
                steps_per_epoch=steps_per_epoch,
                callbacks=[tbCallback, model_checkpoint, earlystopping],
                class_weight=weights,
            )

        hist_df = pd.DataFrame(history.history)
        jsonfile = os.path.join(metrics_path, plane + ".json")

        with open(jsonfile, mode="w") as f:
            hist_df.to_json(f)

    return history




In [ ]:
root_dir = '/workspace/tf/HDDLinux/Stanford/data/mriqc-shared/experiments/experiment_defaced_reproduce/128'

for fold in range(1, 12):
    csv_path = os.path.join(root_dir, f'csv_F15/train_test_fold_{fold}')
    model_save_path = os.path.join(root_dir, f'model_save_dir_F15_shuffled_3/train_test_fold_{fold}')
    tfrecords_path = os.path.join(root_dir, f'tfrecords_F15_shuffled_2/train_test_fold_{fold}')
    history = train(csv_path, model_save_path, tfrecords_path)